#IQPE + Stress-testing

This runs IQPE: the “one-ancilla” version of phase estimation. We apply controlled
𝑈
2
𝑘
U
2
k
, measure the ancilla, use that result to rotate away the phase we already learned, and repeat. The system collapses toward the right eigenstate as we go, and the measured bits build up the phase in binary. In the end we turn that phase back into an energy.

## Installs

In [1]:
!pip install qiskit qiskit-aer qiskit[visualization] matplotlib pylatexenc

In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.synthesis import SuzukiTrotter

## IQPE + Projection

In [8]:
"""
Each iteration:
We entangle ancilla + system using controlled-U and feedback.
We compute the ancilla’s probabilities P(0), P(1).
We pretend we measured ancilla in Z, picking the most likely outcome.
We throw away the other branch of the wavefunction by slicing the statevector and renormalizing.
We continue with just the collapsed system state.
"""


def project_ancilla(state: Statevector, bit: int, system_qubits: int) -> Statevector:
    reshaped = state.data.reshape(2, 2 ** system_qubits)
    projected = reshaped[bit, :]
    norm = np.linalg.norm(projected)
    if norm == 0:
        return Statevector(projected)
    return Statevector(projected / norm)


def iqpe_projected(U, initial_state, num_bits: int, seed=None, verbose=True):
    psi = Statevector(initial_state)
    n_sys = psi.num_qubits

    bits = []
    omega_coef = 0.0

    # Iterate from most significant bit down to least
    for k in range(num_bits, 0, -1):
        omega_coef /= 2.0
        power = 2 ** (k - 1)

        qc = QuantumCircuit(1 + n_sys)
        # 1. Ancilla in |+>
        qc.h(0)

        # 2. Controlled-U^(2^{k-1})
        U_power = U.power(power)
        qc.append(U_power.control(1), [0] + list(range(1, n_sys + 1)))

        # 3. Feedback phase
        theta = -2 * np.pi * omega_coef
        if abs(theta) > 1e-12:
            qc.p(theta, 0)

        # 4. Final H on ancilla
        qc.h(0)

        # 5. Evolve |0>_anc ⊗ |psi>
        full_state = Statevector.from_label("0").tensor(psi)
        full_state = full_state.evolve(qc)

        # 6. Get ancilla marginal probabilities
        probs = np.asarray(full_state.probabilities([0]), dtype=float)
        s = probs.sum()
        if s <= 0 or not np.isfinite(s):
            p0, p1 = 0.5, 0.5
        else:
            p0, p1 = probs / s

        # 7. Majority vote (deterministic)
        bit = 1 if p1 > p0 else 0

        # >>> these two lines were missing <<<
        bits.append(bit)
        omega_coef += bit / 2.0

        if verbose:
            print(f"Iteration k={k}: P(0)={p0:.4f}, P(1)={p1:.4f}, measured={bit}")

        # 8. Project system onto post-measurement state
        psi = project_ancilla(full_state, bit, n_sys)

    phi_est = omega_coef % 1.0
    return phi_est, bits


def unwrap_phase(phi_wrapped: float) -> float:
    """Map a phase in [0,1) to (-0.5, 0.5] for energy reconstruction."""
    return phi_wrapped if phi_wrapped <= 0.5 else phi_wrapped - 1.0


def energy_from_phase(phi_wrapped: float, t_evolution: float) -> float:
    """
    Given φ in [0,1) for U = exp(-i H t),
    φ ≈ -E t / (2π) mod 1 → E ≈ -2π * φ_unwrapped / t.
    """
    phi_cont = unwrap_phase(phi_wrapped)
    return -2 * np.pi * phi_cont / t_evolution

## Testing

In [14]:
# H = 0.33 ZI + 3.24 IZ + 1.17 ZZ
H = SparsePauliOp(["ZI", "IZ", "ZZ"], coeffs=[0.33, 3.24, 1.17])

H_mat = H.to_matrix()
evals, evecs = np.linalg.eigh(H_mat)
print("Exact eigenvalues (ascending):", evals)

# Ground state is |01> with E = -4.08
E_exact = -4.08
eigenstate_index = 0  # if you want, you can also use evecs[:, 0] explicitly

precision_bits = 7
t_evolution = 2 * np.pi / (2**precision_bits)

phi_exact = (-E_exact * t_evolution) / (2 * np.pi) % 1.0
print(f"\nTarget eigenvalue (ground): E_exact = {E_exact:.6f}")
print(f"Evolution time t = {t_evolution:.6f}")
print(f"Exact wrapped phase φ_exact = {phi_exact:.10f}\n")

# Build U = exp(-i H t)
U_gate = PauliEvolutionGate(H, time=t_evolution, synthesis=SuzukiTrotter(reps=2))

# Prepare the exact ground state using the eigenvector from diagonalization
initial_state = evecs[:, 0]

phi_est, bits = iqpe_projected(
    U=U_gate,
    initial_state=initial_state,
    num_bits=precision_bits,
    seed=42,
    verbose=True,
)

E_est = energy_from_phase(phi_est, t_evolution)

print("\n" + "=" * 60)
print("IQPE COMPARISON: Exact vs Estimated")
print("=" * 60)
print(f"Exact wrapped phase φ_exact: {phi_exact:.10f}")
print(f"Estimated phase φ_est:       {phi_est:.10f}")
print(f"Phase error (wrapped):       {abs(phi_est - phi_exact):.10f}")
print()
print(f"Exact eigenvalue E_exact:    {E_exact:.10f}")
print(f"Estimated eigenvalue E_est:  {E_est:.10f}")
print(f"Energy error:                {abs(E_est - E_exact):.10f}")
print()
print(f"Measured bits (k=t..1):      {bits}")
print(f"Binary phase ~ 0.{''.join(map(str, reversed(bits)))} (MSB-first)")


Exact eigenvalues (ascending): [-4.08 -2.4   1.74  4.74]

Target eigenvalue (ground): E_exact = -4.080000
Evolution time t = 0.049087
Exact wrapped phase φ_exact = 0.0318750000

Iteration k=7: P(0)=0.8423, P(1)=0.1577, measured=0
Iteration k=6: P(0)=0.9704, P(1)=0.0296, measured=0
Iteration k=5: P(0)=0.1998, P(1)=0.8002, measured=1
Iteration k=4: P(0)=0.5000, P(1)=0.5000, measured=0
Iteration k=3: P(0)=0.5000, P(1)=0.5000, measured=0
Iteration k=2: P(0)=0.5000, P(1)=0.5000, measured=0
Iteration k=1: P(0)=0.5000, P(1)=0.5000, measured=0

IQPE COMPARISON: Exact vs Estimated
Exact wrapped phase φ_exact: 0.0318750000
Estimated phase φ_est:       0.0312500000
Phase error (wrapped):       0.0006250000

Exact eigenvalue E_exact:    -4.0800000000
Estimated eigenvalue E_est:  -4.0000000000
Energy error:                0.0800000000

Measured bits (k=t..1):      [0, 0, 1, 0, 0, 0, 0]
Binary phase ~ 0.0000100 (MSB-first)
